In [1]:
%load_ext autoreload
%autoreload 2
%pip install prettytable

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import basic package
import os
import pandas as pd
import warnings
import json

warnings.filterwarnings("ignore")
# import the necessary package
from baseline.PyOD import PYOD
from baseline.OE_GPLVM.aeb_gplvm import AEB_GPLVM, NNEncoder
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import VariationalELBO
from gpytorch.priors import NormalPrior, MultivariateNormalPrior
from utils.data_generator import DataGenerator
from utils.myutils import Utils
import matplotlib.pyplot as plt

plt.style.use("ggplot")
import torch
import numpy as np
from tqdm import trange

datagenerator = DataGenerator()  # data generator
utils = Utils()  # utils function

2023-07-10 05:50:25.072876: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 05:50:25.116906: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
# dataset and model list / dict
dataset_list = [
    "01_ALOI",
    "02_annthyroid",
    "03_backdoor",
    "04_breastw",
    "05_campaign",
    "06_cardio",
    "07_Cardiotocography",
    "08_celeba",
    "09_census",
    "99_linear",
]

In [4]:
dataset = dataset_list[5]
datagenerator.dataset = dataset
data = datagenerator.generator(la=1.0, realistic_synthetic_mode=None, noise_type=None)

current noise type: None
{'Samples': 1831, 'Features': 21, 'Anomalies': 176, 'Anomalies Ratio(%)': 9.61}


In [5]:
Y_train = torch.tensor(data["X_train"], dtype=torch.float32)
Y_test = torch.tensor(data["X_test"], dtype=torch.float32)
lb_train = torch.tensor(data["y_train"], dtype=torch.float32)
lb_test = torch.tensor(data["y_test"], dtype=torch.float32)

In [6]:
model_dict = {}
noise_trace_dict = {}
N = len(Y_train)
data_dim = Y_train.shape[1]
latent_dim = 2
n_inducing = 25
batch_size = 100
n_epochs = 1000
nn_layers = (10, 5)
lr = 0.01

# Define prior for X, Initialise model, likelihood, elbo and optimizer and latent variable encoder
X_prior_mean = torch.zeros(N, latent_dim)  # shape: N x Q
prior_x = MultivariateNormalPrior(X_prior_mean, torch.eye(X_prior_mean.shape[1]))
encoder = NNEncoder(N, latent_dim, prior_x, data_dim, layers=nn_layers)
model = AEB_GPLVM(N, data_dim, latent_dim, n_inducing, encoder, nn_layers=nn_layers)
likelihood = GaussianLikelihood()
optimizer = torch.optim.Adam(
    [{"params": model.parameters()}, {"params": likelihood.parameters()}], lr
)
elbo = VariationalELBO(likelihood, model, num_data=len(Y_train), combine_terms=False)

In [7]:
# Model params
print(f'Training model params for model {"GPLVM"}')
model.get_trainable_param_names()
loss_list = []
noise_trace = []

Training model params for model GPLVM
+-----------------------------------------------------------------------+------------+
|                                Modules                                | Parameters |
+-----------------------------------------------------------------------+------------+
|                  variational_strategy.inducing_points                 |     50     |
|    variational_strategy._variational_distribution.variational_mean    |    525     |
| variational_strategy._variational_distribution.chol_variational_covar |   13125    |
|                          X.mu_layers.0.weight                         |    210     |
|                           X.mu_layers.0.bias                          |     10     |
|                          X.mu_layers.1.weight                         |     50     |
|                           X.mu_layers.1.bias                          |     5      |
|                          X.mu_layers.2.weight                         |     10     |
|    

In [21]:
# Provando o conceito
batch_idx_a = np.arange(1, 30)
batch_idx_n = np.arange(32, 100)
batch_index = np.hstack([batch_idx_a, batch_idx_n])

In [22]:
ratio = len(batch_idx_a) / len(batch_index)

In [23]:
output_batch_a = model(model.sample_latent_variable(Y_train)[batch_idx_a])
output_batch_n = model(model.sample_latent_variable(Y_train)[batch_idx_n])
output_batch_full = model(model.sample_latent_variable(Y_train)[batch_index])

In [24]:
# O método forward da classe VariationalELBO, chama a superclasse _ApproximateMarginalLogLikelihood
# que retorna log_likelihood - kl_divergence + log_prior - added_loss, quando você pede pra combinar os termos
# que  é o PADRÃO. Se você passar combine_terms = False, ele te retorna uma tupla com cada um deles
log_likelihood, kl_divergence, log_prior, added_loss = elbo(
    output_batch_full, Y_train[batch_index].T
)
terms = dict(
    log_likelihood=log_likelihood,
    kl_divergence=kl_divergence,
    log_prior=log_prior,
    added_loss=added_loss,
)
terms

{'log_likelihood': tensor([-1.4255, -1.2941, -1.2543, -1.3340, -1.2730, -1.2358, -1.2503, -1.4080,
         -1.2733, -1.2635, -1.2828, -1.4098, -1.3577, -1.3551, -1.3047, -1.2548,
         -1.5054, -1.4636, -1.4630, -1.2519, -1.6503], grad_fn=<DivBackward0>),
 'kl_divergence': tensor([8.9337e-09, 7.4448e-09, 7.4448e-09, 9.6782e-09, 5.2113e-09, 1.0423e-08,
         9.6782e-09, 1.1912e-08, 8.9337e-09, 1.1167e-08, 5.9558e-09, 5.9558e-09,
         1.0423e-08, 1.1167e-08, 6.7003e-09, 1.2656e-08, 8.1892e-09, 8.9337e-09,
         9.6782e-09, 9.6782e-09, 8.1892e-09], grad_fn=<DivBackward0>),
 'log_prior': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'added_loss': tensor([0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748,
         0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748,
         0.1748, 0.1748, 0.1748], grad_fn=<AddBackward0>)}

In [25]:
# Minha duvida era se eu conseguia separar as likelihoods e voltar para o resultado original
# por conta dos termos KL não era trivial, a principio fazer isso
log_likelihood, kl_divergence, log_prior, added_loss = elbo(
    output_batch_n, Y_train[batch_idx_n].T
)
terms = dict(
    log_likelihood=log_likelihood,
    kl_divergence=kl_divergence,
    log_prior=log_prior,
    added_loss=added_loss,
)
terms

{'log_likelihood': tensor([-1.4240, -1.2976, -1.2471, -1.3269, -1.2807, -1.2358, -1.2509, -1.4011,
         -1.2753, -1.2597, -1.2855, -1.4197, -1.3407, -1.3540, -1.3071, -1.2590,
         -1.5090, -1.4674, -1.4669, -1.2476, -1.6921], grad_fn=<DivBackward0>),
 'kl_divergence': tensor([8.9337e-09, 7.4448e-09, 7.4448e-09, 9.6782e-09, 5.2113e-09, 1.0423e-08,
         9.6782e-09, 1.1912e-08, 8.9337e-09, 1.1167e-08, 5.9558e-09, 5.9558e-09,
         1.0423e-08, 1.1167e-08, 6.7003e-09, 1.2656e-08, 8.1892e-09, 8.9337e-09,
         9.6782e-09, 9.6782e-09, 8.1892e-09], grad_fn=<DivBackward0>),
 'log_prior': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'added_loss': tensor([0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748,
         0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748,
         0.1748, 0.1748, 0.1748], grad_fn=<AddBackward0>)}

In [26]:
# Aparentemente os termos KL e Added Loss que também é um KL, são constantes nas chamadas do elbo
# então posso somente armazena-los e focar na log_likelihood
log_likelihood, kl_divergence, log_prior, added_loss = elbo(
    output_batch_n, Y_train[batch_idx_n].T
)
terms = dict(
    log_likelihood=log_likelihood,
    kl_divergence=kl_divergence,
    log_prior=log_prior,
    added_loss=added_loss,
)
terms

{'log_likelihood': tensor([-1.4240, -1.2976, -1.2471, -1.3269, -1.2807, -1.2358, -1.2509, -1.4011,
         -1.2753, -1.2597, -1.2855, -1.4197, -1.3407, -1.3540, -1.3071, -1.2590,
         -1.5090, -1.4674, -1.4669, -1.2476, -1.6921], grad_fn=<DivBackward0>),
 'kl_divergence': tensor([8.9337e-09, 7.4448e-09, 7.4448e-09, 9.6782e-09, 5.2113e-09, 1.0423e-08,
         9.6782e-09, 1.1912e-08, 8.9337e-09, 1.1167e-08, 5.9558e-09, 5.9558e-09,
         1.0423e-08, 1.1167e-08, 6.7003e-09, 1.2656e-08, 8.1892e-09, 8.9337e-09,
         9.6782e-09, 9.6782e-09, 8.1892e-09], grad_fn=<DivBackward0>),
 'log_prior': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'added_loss': tensor([0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748,
         0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748, 0.1748,
         0.1748, 0.1748, 0.1748], grad_fn=<AddBackward0>)}

In [27]:
# Essa é o método principal da classe VariationalELBO, o problem é que ele já vem somado no shape 1xDIM_DADOS
# mas preciso saber esse termo por cada ponto do batch/teste
terms = {
    "_log_likelihood_term": elbo._log_likelihood_term(
        output_batch_full, Y_train[batch_index].T
    )
}
terms

{'_log_likelihood_term': tensor([-138.2719, -125.5316, -121.6715, -129.4010, -123.4836, -119.8682,
         -121.2772, -136.5764, -123.5119, -122.5549, -124.4319, -136.7476,
         -131.6989, -131.4469, -126.5548, -121.7119, -146.0234, -141.9696,
         -141.9101, -121.4336, -160.0825], grad_fn=<SumBackward1>)}

In [28]:
# Eu posso pegar toda a matriz, utilizando a chamada da likelihood
exp_log_prob = elbo.likelihood.expected_log_prob(
    Y_train[batch_index].T, output_batch_full
)
print(exp_log_prob, exp_log_prob.shape)

tensor([[-1.5744, -1.5385, -1.4896,  ..., -1.3072, -1.4575, -1.5563],
        [-1.2460, -1.2695, -1.6928,  ..., -1.2373, -1.2393, -1.3712],
        [-1.2442, -1.2358, -1.2358,  ..., -1.2358, -1.2358, -1.2358],
        ...,
        [-1.5505, -1.5773, -1.5512,  ..., -1.4005, -1.4849, -1.5683],
        [-1.2626, -1.2363, -1.2360,  ..., -1.2358, -1.2358, -1.2359],
        [-1.4161, -1.9567, -1.4163,  ..., -1.4162, -1.4167, -1.9572]],
       grad_fn=<MulBackward0>) torch.Size([21, 97])


In [29]:
# LEMBRANDO QUE ---> exp_log_prob.sum(-1).div(batch_size) == elbo(target = Y_train[batch_index].T, variational_dist_f= output_batch_full)[0]
## IMPORTANTE!!! Se atentar aos gradienets, basta multiplicar exp_log_prob.sum(-1).div(batch_size) por *1 para ver o B.O.
batch_size = len(batch_index)
exp_log_prob.sum(-1).div(batch_size)

tensor([-1.4255, -1.2941, -1.2543, -1.3340, -1.2730, -1.2358, -1.2503, -1.4080,
        -1.2733, -1.2635, -1.2828, -1.4098, -1.3577, -1.3551, -1.3047, -1.2548,
        -1.5054, -1.4636, -1.4630, -1.2519, -1.6503], grad_fn=<DivBackward0>)

In [30]:
# LL TOTAL
log_likelihood_total = elbo(
    target=Y_train[batch_index].T, variational_dist_f=output_batch_full
)[0]

# Depois de muito sofrer pensando isso, percebi que posso fazer passar chamar o elbo
# pra cada conjunto, anomalo e normal, e depois recuperar o log_likelihood total
# se eu fizer uma soma ponderada dos log_likelihoods
# Minha preocupacao aqui era por que na instaciação da classe você precisa
# passar o num_data, só que esse termo se não diz respeito ao tamanno do batch
# e sim ao tamanho do treino total. Em todos os experimentos do GPLVM
# ele é instanciado utilizado num_data = len(Y_train), logo isso não interfere
# no calculo do elbo. Também é legal ressaltar que os termos KL não mudam
# mesmo que os batchs mudem.

output_batch_a = model(model.sample_latent_variable(Y_train)[batch_idx_a])
output_batch_n = model(model.sample_latent_variable(Y_train)[batch_idx_n])
target_a = Y_train[batch_idx_a].T
target_n = Y_train[batch_idx_n].T
ll_normal = elbo(output_batch_n, target_n)[0] * (
    1 - ratio
)  # <- Isso não é o elbo. É a log_likelihood --> Se atentar ao [0]
ll_anomaly = elbo(output_batch_a, target_a)[0] * ratio

In [33]:
log_likelihood_total

tensor([-1.4255, -1.2941, -1.2543, -1.3340, -1.2730, -1.2358, -1.2503, -1.4080,
        -1.2733, -1.2635, -1.2828, -1.4098, -1.3577, -1.3551, -1.3047, -1.2548,
        -1.5054, -1.4636, -1.4630, -1.2519, -1.6503], grad_fn=<DivBackward0>)

In [34]:
ll_normal + ll_anomaly

tensor([-1.4254, -1.2942, -1.2543, -1.3340, -1.2730, -1.2358, -1.2503, -1.4080,
        -1.2733, -1.2634, -1.2828, -1.4098, -1.3577, -1.3551, -1.3047, -1.2548,
        -1.5054, -1.4637, -1.4630, -1.2519, -1.6504], grad_fn=<AddBackward0>)

In [ ]:
def get_loe_index(score, ratio, batch_index):
    _, idx_n = torch.topk(
        score, int(score.shape[0] * (1 - ratio)), largest=False, sorted=False
    )
    _, idx_a = torch.topk(
        score, int(score.shape[0] * (ratio)), largest=True, sorted=False
    )
    return batch_index[idx_n], batch_index[idx_a]

In [ ]:
def get_log_likelihood_LOE(Y_train, lb_train, idx_n, idx_a, ratio, method="blind"):
    output_batch_a = model(model.sample_latent_variable(Y_train)[idx_a])
    output_batch_n = model(model.sample_latent_variable(Y_train)[idx_n])
    target_a = Y_train[idx_a].T
    target_n = Y_train[idx_n].T

    ll_normal = elbo(output_batch_n, target_n)[0] * (1 - ratio)
    ll_anomaly = elbo(output_batch_a, target_a)[0] * ratio

    if method == "blind":
        log_likelihood = (ll_normal + ll_anomaly).div(1)
    elif method == "hard":
        log_likelihood = (ll_normal - ll_anomaly).div(1)
    elif method == "soft":
        log_likelihood = (ll_normal - 0.5 * (ll_anomaly + ll_normal)).div(1)
    else:
        raise NotImplementedError

    return log_likelihood

In [ ]:
def train(model, batch_size, Y_train, y_train, method="refine"):
    """
    No caso blind, nos treinamos tudo como se fosse normal, inclusive as anomalias
    """
    """
    Preciso avaliar se o ratio é > 0, caso não seja, preciso selecionar apenas indices de elementos normais
    esse é o método do refine. 
    Segundo  avalio o score de cada ponto normalmente e retorno a log-likelihood de cada termo
    """
    if method == "refine":
        batch_index = model._get_normal_batch_idx(batch_size, y_train)
        output_batch = model(model.sample_latent_variable(Y_train)[batch_index])

        log_likelihood, kl_divergence, log_prior, added_loss = elbo(
            output_batch, Y_train[batch_index].T
        )

    elif method in ["hard", "soft", "blind"]:
        batch_index, ratio = model._get_individual_batch_idx(batch_size, y_train)
        output_batch = model(model.sample_latent_variable(Y_train)[batch_index])

        _, kl_divergence, log_prior, added_loss = elbo(
            output_batch, Y_train[batch_index].T
        )

        likelihood_per_point = elbo.likelihood.expected_log_prob(
            Y_train[batch_index].T, output_batch
        ).sum(0)
        loe_idx_n, loe_idx_a = get_loe_index(likelihood_per_point, ratio, batch_index)
        log_likelihood = get_log_likelihood_LOE(
            Y_train, y_train, loe_idx_n, loe_idx_a, ratio
        )

    loss = -(log_likelihood - kl_divergence + log_prior - added_loss).sum()

    return loss

In [ ]:
train(model, batch_size, Y_train, lb_train, method="refine")

In [ ]:
get_log_likelihood_term(Y_train, lb_train, loe_idx_n, loe_idx_a)

In [ ]:
get_log_likelihood_term(Y_train, lb_train, idx_n, idx_a)

In [ ]:
batch_index = np.append(idx_n, idx_a)
output_batch = model(model.sample_latent_variable(Y_train)[idx_n])
log_likelihood, kl_divergence, log_prior, added_loss = elbo(
    output_batch, Y_train[idx_n].T
)
-(log_likelihood - kl_divergence + log_prior - added_loss).sum()

In [ ]:
elbo(output_batch, Y_train[batch_index].T)

In [ ]:
ll, kl_divergence, log_prior, added_loss = elbo(
    output_batch_full, Y_train[batch_index].T
)

In [ ]:
get_log_likelihood_term(Y_train, lb_train, batch_index)

In [ ]:
iterator = trange(1000, leave=True)
batch_size = 100
for i in iterator:
    optimizer.zero_grad()
    loss = train(model, batch_size, Y_train, lb_train, method="hard")
    loss_list.append(loss.item())
    noise_trace.append(np.round(likelihood.noise_covar.noise.item(), 3))
    if i % 100 == 0:
        iterator.set_description(
            "Loss: " + str(float(np.round(loss.item(), 2))) + ", iter no: " + str(i)
        )
    loss.backward()
    optimizer.step()

model.store(loss_list, likelihood)
noise_trace_dict["GPLVM"] = noise_trace
X_train_mean = model.get_X_mean(Y_train)

In [ ]:
with torch.no_grad():
    model.eval()
    likelihood.eval()

X_test_mean, X_test_covar = model.predict_latent(Y_test)
Y_test_mean, Y_test_covar = model.reconstruct_y(Y_test)

## amostra -> nn_ecoder --> model.decoder --> amostra_reconstruida
## ou seja, vira uma variavel latente, depois passa pelo modelo
## para se tornar de novo algo no espaço dos dados

In [ ]:
def scatter_Y_test(Y_test, lb_test):
    idx_n = np.where(lb_test == 0)
    idx_a = np.where(lb_test == 1)
    plt.scatter(Y_test_mean[0, idx_a], Y_test_mean[1, idx_a])
    plt.scatter(Y_test_mean[0, idx_n], Y_test_mean[1, idx_n])

In [ ]:
scatter_Y_test(Y_test, lb_test)

In [ ]:
# seed for reproducible results
seed = 42

# clf = clf(seed=seed, model_name=name)
# clf = clf.fit(X_train=data["X_train"], y_train=data["y_train"])
# score = clf.predict_score(data["X_test"])
# result = utils.metric(y_true=data["y_test"], y_score=score)
# df_AUCROC.loc[dataset, name] = result["aucroc"]
# df_AUCPR.loc[dataset, name] = result["aucpr"]